In [2]:
import numpy as np
from pyballistics import ozvb_lagrange, get_db_powder, get_powder_names

In [3]:
from modules import Cannon, get_parametr_1d, get_parametr_matrix, get_time
from modules.wave_long import solver, solver_wave

In [4]:
import matplotlib.pyplot as plt

In [5]:
import scipy.interpolate as interp
from scipy.interpolate import splrep, BSpline
from scipy.signal import savgol_filter
from findiff import FinDiff, pde, Coef, BoundaryConditions, PDE

# 1. Ballistic

In [6]:
d = 125*1e-3          # калибр м
q = 7.05           # вес снаряда кг
velocity_pm = 1700         # дульная скорость снаряда
n_s = 1           # нарезное орудие
max_pressure = 600*1e6    # максимальное давление Па
tube_lenght = 5543*1e-3      # длина трубы     
p_fors = 10*1e6

pow_name = "МАП-1 23/1"
wq, ro = 1.178537, 737.374805

In [7]:
init_dict = {
   'powders': 
    [
       {'omega': wq*q,
       'dbname': pow_name}
    ],
    'init_conditions': 
    {
       'q': q,
       'd': d,
       'W_0': wq*q/ro,
       'phi_1': 1.0,
       'p_0': p_fors, 
       'n_S': n_s
    },
    'igniter': 
    {
       'p_ign_0': 5_000_000.0 #check it 
    },
  
    'meta_lagrange': 
    {
       'CFL': 0.9, 
       'n_cells': 300
    },
    'stop_conditions': 
    {
       'x_p': tube_lenght,
       'steps_max': 8000,   
       't_max': 0.08,
       'p_max': max_pressure,
    }
    }

In [8]:
ballistic =  ozvb_lagrange(init_dict)

In [9]:
print ('stop= ', ballistic['stop_reason'], "|  x_0=", ballistic['layers'][-1]['x'][0], 
       "l_tube=", ballistic['layers'][-1]['x'][-1], 'v=', ballistic['layers'][-1]['u'][-1])

stop=  x_p |  x_0= -0.9181933370995707 l_tube= 5.543 v= 1725.9951133206675


In [10]:
arr_time = get_time(ballistic)
arr_x_pr = get_parametr_1d(ballistic, 'x')
arr_p_pr = get_parametr_1d(ballistic, 'p')
matrix_x = get_parametr_matrix(ballistic, 'x')
matrix_p = get_parametr_matrix(ballistic, 'p')
arr_p_mean = np.mean(matrix_p, axis=1)

#add layre to 'p' for shape alignment with 'x'
matrix_p = np.row_stack((matrix_p.T, matrix_p.T[-1])).T
#calculate coordinate from 0 point
l0 = np.abs(matrix_x[0][0])
matrix_x += l0

In [11]:
matrix_p.shape, matrix_x.shape, arr_p_mean.shape, arr_time[-1]*1000

((1888, 301), (1888, 301), (1888,), 7.916475936787449)

# 2. Geometry

In [12]:
rho = 7800
E = 200e9
vi = 0.25
g = 9.81

# geometry grid
n_x = 500

In [13]:
Cannon.n = n_x
geometry = Cannon(d, matrix_x, matrix_p, l0)
geometry.cannon_geometry()

In [14]:
beam_coord = geometry.coordinate
beam_r1 = geometry.r_inside
beam_r1_x = geometry.r_inside_coordinate

In [15]:
l_kam = beam_r1_x[-2]

In [16]:
beam_r2_x = np.array([0, 0.4, 0.5, 0.8, beam_coord[-1]])
beam_r2 = np.array([0.16, 0.16, 0.16, 0.14, 0.09])

In [17]:
matrix_p_xt = geometry.make_p_xt()

In [18]:
matrix_p_xt.shape

(1888, 500)

### Make interpolate

In [19]:
# r1(x) and r2(x)
func_r1 = interp.interp1d(beam_r1_x, beam_r1, bounds_error=False, fill_value=(0, 0))
func_r2 = interp.interp1d(beam_r2_x, beam_r2, bounds_error=False, fill_value=(0, 0))

In [20]:
# p(t, x)
arr_x = np.linspace(beam_coord[0], beam_coord[-1], n_x)
func_p_xt = interp.RegularGridInterpolator((arr_time, arr_x), matrix_p_xt)

In [21]:
# p_mean(t)
func_p_mean = interp.interp1d(arr_time, arr_p_mean, bounds_error=False, fill_value=(0, 0))
# x_dn(t)
arr_x_pr += l_kam
func_x_pr = interp.interp1d(arr_time, arr_x_pr, bounds_error=False, fill_value=(0, 0))

### Prepare function

In [22]:
fun_F = lambda x: np.pi*(func_r2(x)**2 - func_r1(x)**2)
fun_S = lambda x: np.pi*func_r1(x)**2
fun_J = lambda x: np.pi/4 * (func_r2(x)**4 - func_r1(x)**4)
fun_p_pr = lambda t: func_p_mean(t) /(1 + 1/3 * wq)
fun_p_kn = lambda t: 1+0.5*wq*fun_p_pr(t)

def fun_p (t, x):
    x_pr = func_x_pr(t)
    if x > x_pr or x < 0:
        return 0
    else:
        return fun_p_kn(t) - 0.5*wq*fun_p_pr(t)* x**2/x_pr**2

def fun_dp_dx(t, x):
    x_pr = func_x_pr(t)
    if x > x_pr or x < 0:
        return 0
    else:
        return - wq*fun_p_pr(t)*x/x_pr**2
    
fun_p = np.vectorize(fun_p)
fun_dp_dx = np.vectorize(fun_dp_dx)

# 3. Wave x (long)

In [23]:
q_x = lambda t, x: 0
displ = arr_x[-1] - 5 # положение заделки
n_t = 300

### Prepare f(t, x)

In [24]:
# Define dS/dx:
dx = arr_x[1] - arr_x[0]
d_dx = FinDiff(0, dx, 1)
dS_dx = d_dx(fun_S(arr_x))

fun_dS_dx = interp.interp1d(arr_x, dS_dx, bounds_error=False, fill_value=(0, 0))

In [25]:
def f_i(x, t, del_x = displ):
    x_new = x + del_x
    return 2*vi*(fun_p(t, x_new)*fun_dS_dx(x_new) + \
                 fun_S(x_new)*fun_dp_dx(t, x_new)) - \
                fun_p(t, x_new)*fun_dS_dx(x_new) - q_x(t, x_new)


In [26]:
F_mean = np.mean(fun_F(np.linspace(displ, arr_x[-1], n_x)))
F_mean

0.02802050420709615

### Solve euption

In [88]:
# u_tt = a*(c**2*u_x)_x + a*f(x,t) on (0,L)
# начальные условия 
I = lambda x: 0 # u(0, x)
V = lambda x: 0 # du/dt (t=0)
# граничные условия 
U_0 = lambda t: 0 # u(t, 0) if None - du/dx = 0 use - в начале 
U_L = None # u(t, L) if None - du/dx = 0 use - на конце

# функция f(x, t)
# Если решаем на учатске F и S - const, то делим на rho и F
f = lambda x, t: f_i(x, t) # f(x, t)
a = lambda x: 1/rho/fun_F(x)
c = lambda x: np.sqrt(E*fun_F(x))

dt = arr_time[-1]/n_t
T = arr_time[-1]-dt # time stop

L = arr_x[-1] - displ
C = 0.75 # the Courant number (=max(c)*dt/dx).

In [89]:
%%time
grid_x, grid_t, u_matrix = solver_wave(I, V, f_t, c_f, U_0, U_L, L, dt, C, T, a_f, version='vectorized')

N_x =  28
CPU times: total: 1.14 s
Wall time: 1.21 s


In [90]:
u_matrix = np.array(u_matrix)

In [91]:
u = u_matrix.copy()

### Vizualization

In [92]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [93]:
#%matplotlib inline 
#%config InlineBackend.figure_format = 'svg' 
rc = {"font.family" : "serif", 
      "mathtext.fontset" : "stix"}
plt.rcParams.update(rc)
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]

In [95]:
plt.ioff()

step_slider = widgets.IntSlider(
    orientation='horizontal',
    value=0, 
    min=0, max=len(grid_t), step=1, 
    description='step'
)

step_slider.layout.margin = '0px 30% 0px 30%'
step_slider.layout.width = '40%'

fig = plt.figure(figsize=(10,6))
fig.canvas.header_visible = False
fig.canvas.layout.min_height = '400px'
#plt.title('Plotting step: {}'.format(data_x[step_slider.value]['t_s']))

line1 = plt.plot(grid_x, u[0]*1e6, lw=1)

plt.xlabel('$x$, м', fontsize=16)
plt.ylabel('$u$, мкм', fontsize=16)
plt.xlim(0, 6)
plt.ylim(-300, 300)
plt.grid()


def update_lines(change):
    #plt.title('Plotting step: {}'.format(data_x[step_slider.value]['t_s']))
    line1[0].set_data(grid_x, u[change.new]*1e6)
    fig.canvas.draw()
    fig.canvas.flush_events()

step_slider.observe(update_lines, names='value')

widgets.AppLayout(
    center=fig.canvas,
    footer=step_slider,
    pane_heights=[0, 6, 1]
)

AppLayout(children=(IntSlider(value=0, description='step', layout=Layout(grid_area='footer', margin='0px 30% 0…